In [3]:
import pandas as pd 
from uszipcode import SearchEngine
import sqlalchemy
from sqlalchemy import create_engine, types
from tqdm import tqdm

from config import postgre_pw


In [5]:
#read AirBNB dataset

airbnb_df = pd.read_csv("data/airbnb.csv")
airbnb_filtered__df = airbnb_df[airbnb_df["number_of_reviews"] > 25]
airbnb_filtered__df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
15,74404.0,Luxury 3 bed/ 2 bath apt in Harlem w/ terrace,391325.0,G & S,Manhattan,Harlem,40.80276,-73.95670,Entire home/apt,250.0,14.0,31.0,8/22/2012,0.31,1.0,78.0
17,47362.0,"LARGE, COMFY 1BDR W/CHARACTER!!!",214148.0,Robert,Brooklyn,Bedford-Stuyvesant,40.68237,-73.94150,Entire home/apt,175.0,26.0,30.0,9/3/2012,0.29,1.0,364.0
25,23501.0,Monkey Retreat Manhattan,63318.0,Meka,Manhattan,Washington Heights,40.83927,-73.94281,Private room,65.0,2.0,68.0,11/1/2012,0.60,1.0,312.0
108,591565.0,Everyone who stays leaves happy!,2919467.0,Lisa,Manhattan,Tribeca,40.71552,-74.00749,Private room,229.0,1.0,62.0,4/27/2014,0.73,1.0,36.0
113,71384.0,Gigantic Private Brooklyn Loft!,365153.0,Ben,Brooklyn,Greenpoint,40.72898,-73.95552,Entire home/apt,229.0,1.0,50.0,5/13/2014,0.50,1.0,188.0


In [6]:
#subset dataset using copy to make a new DF 
airbnb_filtered__df = airbnb_filtered__df[["neighbourhood_group", 
                                           "neighbourhood", 
                                           "latitude",
                                           "longitude",
                                           "room_type",
                                           "price", 
                                           "number_of_reviews",
                                           "last_review"]].copy()
airbnb_filtered__df.rename(columns={"neighbourhood_group":"borough"}, inplace=True)
airbnb_filtered__df.index.names = ['id']
airbnb_filtered__df.head()

,borough,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,last_review
id,,,,,,,,
15,Manhattan,Harlem,40.80276,-73.95670,Entire home/apt,250.0,31.0,8/22/2012
17,Brooklyn,Bedford-Stuyvesant,40.68237,-73.94150,Entire home/apt,175.0,30.0,9/3/2012
25,Manhattan,Washington Heights,40.83927,-73.94281,Private room,65.0,68.0,11/1/2012
108,Manhattan,Tribeca,40.71552,-74.00749,Private room,229.0,62.0,4/27/2014
113,Brooklyn,Greenpoint,40.72898,-73.95552,Entire home/apt,229.0,50.0,5/13/2014


In [7]:
#This function is applied to the entire Pandas dataframe.
#The AirBNB dataset only includes lat, lng coordinates, so we have to query the USZIPCODE library to pull out the associated zipcode,
#and that zipcodes assoicated demographic information. 
#We capture the entire USZIPCODE result object for each coordinate set once, and store that within the individual listing row. 
#The result is returned as a dictionary.
def get_zipcode_data(lat, lng): 
    search = SearchEngine(simple_zipcode=True)
    try:
        result = search.by_coordinates(lat, lng, radius=1, returns=1)
        zipcode_data = result[0].to_dict()
    except:
        zipcode_data = "Failure"
    return zipcode_data

In [8]:
#setup TQDM progress bar
tqdm.pandas(desc="Processing", total = len(airbnb_filtered__df))

#apply get_zipcode_data to dataframe
#use TQDM's progress_apply so that we can see a progress bar of the application.
airbnb_filtered__df["zipcode_data"] = airbnb_filtered__df.progress_apply(lambda row: get_zipcode_data(row["latitude"], row["longitude"]), axis=1)


Processing:: 100%|███████████████████████████████████████████████████████████████| 11617/11617 [02:17<00:00, 84.46it/s]


In [23]:
#Filter out any of the records where the lat,lng to zipcode lookup failed.
airbnb_filtered__df = airbnb_filtered__df[airbnb_filtered__df["zipcode_data"]!="Failure"].copy() 

In [24]:
#This is a general function that pulls information from the complete USZIPCODE object result set. 
#The result set contains a wide varitey of information, so we can select the individual results that we want by pulling out
#the value by looking up the selected key from the result set dictionary. 
def get_key_value(selected_key, zipcode_data):
    selected_key_value = zipcode_data[selected_key]
    return selected_key_value

#get zipcode for each row listing
airbnb_filtered__df["zipcode"] = airbnb_filtered__df.progress_apply(lambda row: get_key_value("zipcode", row["zipcode_data"]), axis=1)

#get median_household_income for zipcode of row listing
airbnb_filtered__df["median_household_income"] = airbnb_filtered__df.progress_apply(lambda row: get_key_value("median_household_income", row["zipcode_data"]), axis=1)

#get median_home_value for zipcode of row listing
airbnb_filtered__df["median_home_value"] = airbnb_filtered__df.progress_apply(lambda row: get_key_value("median_home_value", row["zipcode_data"]), axis=1)


Processing:: 100%|████████████████████████████████████████████████████████████| 11360/11360 [00:00<00:00, 20052.24it/s]

Processing:: 100%|████████████████████████████████████████████████████████████| 11360/11360 [00:00<00:00, 21288.94it/s]

Processing:: 100%|████████████████████████████████████████████████████████████| 11360/11360 [00:00<00:00, 26245.14it/s]


In [28]:
airbnb_filtered__df.head()

,borough,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,last_review,zipcode_data,zipcode,median_household_income,median_home_value
id,,,,,,,,,,,,
15,Manhattan,Harlem,40.80276,-73.95670,Entire home/apt,250.0,31.0,8/22/2012,"{'zipcode': '10026', 'zipcode_type': 'Standard...",10026,43107.0,602400.0
17,Brooklyn,Bedford-Stuyvesant,40.68237,-73.94150,Entire home/apt,175.0,30.0,9/3/2012,"{'zipcode': '11216', 'zipcode_type': 'Standard...",11216,43996.0,623300.0
25,Manhattan,Washington Heights,40.83927,-73.94281,Private room,65.0,68.0,11/1/2012,"{'zipcode': '10032', 'zipcode_type': 'Standard...",10032,34568.0,334900.0
108,Manhattan,Tribeca,40.71552,-74.00749,Private room,229.0,62.0,4/27/2014,"{'zipcode': '10007', 'zipcode_type': 'Standard...",10007,216037.0,1000001.0
113,Brooklyn,Greenpoint,40.72898,-73.95552,Entire home/apt,229.0,50.0,5/13/2014,"{'zipcode': '11222', 'zipcode_type': 'Standard...",11222,63739.0,726500.0


In [26]:
#connect to PostgreSQL Database
connection_string = f"postgres:{postgre_pw}@localhost:5432/AirBNB_NYC"
engine = create_engine(f'postgresql://{connection_string}')

#INSERT the entire df into the sql database.
airbnb_filtered__df.to_sql(name="airbnb_listings",
                          con=engine,
                          if_exists="append",
                          index=True,
                          dtype={'zipcode_data': sqlalchemy.types.JSON}) #we need to set this dtype to JSON so that we can store the dictionary.

In [27]:
#save off CSV with final data 
airbnb_filtered__df.to_csv("data/airbnb__final.csv")